In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
from datetime import date
from datetime import timedelta 

today = date.today()
yesterday = today - timedelta(days = 1) 

def parseHeader(header):
    return([l for l in header.split("\n") if l])

def parseBody(body, header):
    lst = []
    b = body.pop()
    b1 = [l for l in b.split("\n\n") if l]
    for item in b1:
        lst.append([l for l in item.split("\n") if l])
    df = pd.DataFrame(lst, columns =header)
    return df.dropna()
    
def covid19():   
    # Ministry of Health and Family Welfare Website
    url='https://www.mohfw.gov.in/'
    
    resp=requests.get(url) 
    
    # List to contain all unformatted tabular data
    body = []
    
    # Check if website returns valid response
    if resp.status_code==200:  
        
        # Parse received webpage
        soup=BeautifulSoup(resp.text,'html.parser')
        
        # Search for div with class(that contains table data) after webpage inspection
        l=soup.find("div",{"class":"content newtab"}) 
      
        # Retrieve header of the table
        for i in l.findAll("thead"): 
            header = i.text
            
        # Parse the header
        header = parseHeader(header)
        
        # Retrieve the content of the table
        for i in l.findAll("tbody"):
            body.append(i.text)
        # Parse the body and create dataframe
        dataset = parseBody(body, header)
        
        # Export data for future analysis
        dataset.to_json('covidData\\covid19_'+str(today)+".json")
    else: 
        print("Error")
        
def generateMessage(): 
    # Generate Json Data after web scraping
    covid19()
    
    # Variables for storing columnar data
    states = []
    cases = []
    cured = []
    deaths = []
    buffer = []
    
    # Retreive today's data and yesterday's data for comparison
    df1=pd.read_json ('covidData\\covid19_'+str(today)+".json")
    df2=pd.read_json ('covidData\\covid19_'+str(yesterday)+".json")
    
    # Find which rows have updated values and display them
    dff = df2[~df1.apply(tuple,1).isin(df2.apply(tuple,1))]
    
    #Sort Data so that state with highest number of cases shows first
    dff.sort_values("Total Confirmed cases (Indian National)", axis = 0, ascending = False, inplace = True, na_position ='last');
     
    # Start generating the message to be sent
    msg="SARS-COV-2 | New cases as of "+str(today)+"\n\n"+"States with new reported cases :\n\n"
    
    # Iterate over each row and store columnar data
    for _, row in dff.iterrows():
        states.append(row['Name of State / UT'])
        cases.append(row['Total Confirmed cases (Indian National)']+row['Total Confirmed cases ( Foreign National )'])
        cured.append(row['Cured/Discharged/Migrated'])
        deaths.append(row['Death'])
       
    # Generate the final message using stored data
    for i in range(len(dff)):
        buffer.append('State'+": "+str(states[i])+" | "+'Cases'+": "+str(cases[i])+" | "+'Cured'+": "+str(cured[i])+" | "+'Deaths'+": "+str(deaths[i])+"\n\n")
    buffer.append("For more information, visit https://www.mohfw.gov.in/ \n\n")
    
    # Form the final message to be sent
    for i in range(len(buffer)):
        msg+= buffer[i]
    return msg
newMsg = generateMessage()
print(newMsg)

SARS-COV2-19 | New cases as of 2020-03-28

States with new reported cases :

State: Kerala | Cases: 137 | Cured: 11 | Deaths: 0

State: Maharashtra | Cases: 130 | Cured: 15 | Deaths: 4

State: Gujarat | Cases: 43 | Cured: 0 | Deaths: 3

State: Uttar Pradesh | Cases: 41 | Cured: 11 | Deaths: 0

State: Rajasthan | Cases: 41 | Cured: 3 | Deaths: 0

State: Delhi | Cases: 36 | Cured: 6 | Deaths: 1

State: Telengana | Cases: 45 | Cured: 1 | Deaths: 0

State: Punjab | Cases: 33 | Cured: 0 | Deaths: 1

State: Tamil Nadu | Cases: 29 | Cured: 1 | Deaths: 1

State: Madhya Pradesh | Cases: 20 | Cured: 0 | Deaths: 1

State: Haryana | Cases: 30 | Cured: 11 | Deaths: 0

State: Jammu and Kashmir | Cases: 13 | Cured: 1 | Deaths: 1

State: Andhra Pradesh | Cases: 12 | Cured: 1 | Deaths: 0

State: West Bengal | Cases: 10 | Cured: 0 | Deaths: 1

State: Bihar | Cases: 6 | Cured: 0 | Deaths: 1

State: Odisha | Cases: 2 | Cured: 0 | Deaths: 0

State: Andaman and Nicobar Islands | Cases: 1 | Cured: 0 | Deaths

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
from twilio.rest import Client

account_sid = 'AC4d865571fXXXXXXf07b8e1a03e882181'
auth_token = 'ed7707c22660XXXXXX67aab81e1972a1'
client = Client(account_sid, auth_token)

message = client.messages.create(
                              body=newMsg,
                              from_='+120XXXX6825',
                              to='+919XXXX31518'
                          )
print(message.sid)

SMa6455aa1119d4a9787d16b41260f71ca
